In [1]:
import os

import numpy as np
import pandas as pd

import xarray as xr

import xesmf as xe

In [2]:
path_d = "../results/"
path_r = "../temp/NSRDB/"
names = [ "NSRDB_4km.nc", "NSRDB_2km.nc" ]
ds_4 = xr.open_dataset(path_d + names[0]).sel()
ds_2 = xr.open_dataset(path_d + names[1])
ds_4["Temperature"] += 273.15
ds_4["Dew Point"] += 273.15
ds_2["Temperature"] += 273.15
ds_2["Dew Point"] += 273.15
ds_2 = ds_2.drop_vars("Cloud Type")#

In [3]:
# Cambiamos la resolución.
regridder = xe.Regridder( ds_4, ds_2, "conservative" )
ds_2_int = regridder( ds_4, keep_attrs = True )
ds_2_int

<xarray.Dataset> Size: 394MB
Dimensions:                                      (time: 219144, lat: 4, lon: 4)
Coordinates:
  * time                                         (time) datetime64[ns] 2MB 19...
  * lon                                          (lon) float64 32B -99.17 ......
  * lat                                          (lat) float64 32B 19.38 ... ...
Data variables: (12/14)
    Temperature                                  (time, lat, lon) float64 28MB ...
    GHI                                          (time, lat, lon) float64 28MB ...
    DNI                                          (time, lat, lon) float64 28MB ...
    Relative Humidity                            (time, lat, lon) float64 28MB ...
    Pressure                                     (time, lat, lon) float64 28MB ...
    Wind Speed                                   (time, lat, lon) float64 28MB ...
    ...                                           ...
    DHI                                          (time, lat, lon) float64 28MB ...
    Solar Zenith Angle                           (time, lat, lon) float64 28MB ...
    Surface Albedo                               (time, lat, lon) float64 28MB ...
    Precipitable Water                           (time, lat, lon) float64 28MB ...
    Global Horizontal UV Irradiance (280-400nm)  (time, lat, lon) float64 28MB ...
    Global Horizontal UV Irradiance (295-385nm)  (time, lat, lon) float64 28MB ...
Attributes:
    CDI:            Climate Data Interface version 2.4.0 (https://mpimet.mpg....
    Conventions:    CF-1.6
    history:        Tue Apr 02 09:03:52 2024: cdo -O -s collgrid temp/NSRDB/N...
    CDO:            Climate Data Operators version 2.4.0 (https://mpimet.mpg....
    regrid_method:  conservative

In [4]:
v_uv = [ "Global Horizontal UV Irradiance (280-400nm)",
    "Global Horizontal UV Irradiance (295-385nm)" ]

f_int = ds_2 / ds_2_int.drop_vars( v_uv )
f_int = f_int.where( f_int.apply(np.isfinite) | ~f_int.isnull(), 1 )
f_int.mean( "time", skipna = True )
f_int[ v_uv[0] ] = f_int["GHI"]
f_int[ v_uv[1] ] = f_int["GHI"]
ds_2_corr = ds_2_int * f_int
ds_2_corr["UVHI"] = ds_2_corr[ v_uv[0] ] + ds_2_corr[ v_uv[1] ]
ds_2_corr = ds_2_corr.rename_vars( { "Relative Humidity": "Relative_Humidity",
    "Wind Speed": "Wind_Speed", "Wind Direction": "Wind_Direction",
    "Dew Point": "Dew_Point", "Surface Albedo": "Surface_Albedo",
    "Precipitable Water": "Precipitable_Water",
    "Solar Zenith Angle": "Solar_Zenith_Angle",
    v_uv[0]: "UV_A", v_uv[1]: "UV_B" } )

ds_2_corr

<xarray.Dataset> Size: 84MB
Dimensions:             (time: 43824, lon: 4, lat: 4)
Coordinates:
  * time                (time) datetime64[ns] 351kB 2018-01-01 ... 2022-12-31...
  * lon                 (lon) float64 32B -99.17 -99.15 -99.13 -99.11
  * lat                 (lat) float64 32B 19.38 19.4 19.42 19.44
Data variables: (12/15)
    Temperature         (time, lat, lon) float64 6MB 286.8 286.8 ... 289.3 289.4
    GHI                 (time, lat, lon) float64 6MB 0.0 0.0 0.0 ... 81.0 67.0
    DNI                 (time, lat, lon) float64 6MB 0.0 0.0 0.0 ... 53.0 5.0
    Relative_Humidity   (time, lat, lon) float64 6MB 55.7 55.5 ... 52.6 52.2
    Pressure            (time, lat, lon) float64 6MB 786.0 787.0 ... 786.0 786.0
    Wind_Speed          (time, lat, lon) float64 6MB 0.5 0.5 0.5 ... 0.9 0.9 1.0
    ...                  ...
    Solar_Zenith_Angle  (time, lat, lon) float64 6MB 95.3 95.3 ... 82.5 82.5
    Surface_Albedo      (time, lat, lon) float64 6MB 0.1 0.1 0.1 ... 1.7 0.1 0.1
    Precipitable_Water  (time, lat, lon) float64 6MB 1.4 1.4 1.4 ... 1.7 1.7
    UV_A                (time, lat, lon) float64 6MB 0.0 0.0 ... 0.001386 0.0
    UV_B                (time, lat, lon) float64 6MB 0.0 0.0 ... 0.0001802 0.0
    UVHI                (time, lat, lon) float64 6MB 0.0 0.0 ... 0.001566 0.0

In [5]:
ds_2_corr.to_netcdf( path_r + "quantile_prep/NSRDB_int_001.nc" )

vars = list(ds_2_corr.keys())
for i in ["UV_A", "UV_B", "Solar_Zenith_Angle", "Dew_Point", "DHI"]:
    vars.remove(i)

for v in list(ds_2_corr.keys()):
    if not os.path.exists( path_r + "quantile_vars/" + v + "/" ):
        os.mkdir( path_r + "quantile_vars/" + v + "/" )
    ds_2_corr[[v]].to_netcdf( path_r + "quantile_vars/"
        + v + "/NSRDB_int_001.nc" ) 